In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

len(os.listdir('/kaggle/input/vid2img-nfl-impacts/train_images'))
img_path = '/kaggle/input/vid2img-nfl-impacts/train_images/' + os.listdir('/kaggle/input/vid2img-nfl-impacts/train_images')[0]

In [ ]:
img = cv2.imread(img_path)
img.shape

In [ ]:
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
import torch 
from xgboost import XGBClassifier
import xgboost as xgb
# from sklearn.preprocessing import Imputer

In [ ]:
video_labels = pd.read_csv('/kaggle/input/nfl-impact-detection/train_labels.csv').fillna(0)

In [ ]:
video_labels['impact'].value_counts()

In [ ]:
col = list(video_labels.columns.copy())

In [ ]:
col.remove('impact')
col.remove('impactType')

In [ ]:
x = video_labels[['left','width','top','height']]

In [ ]:
df = pd.read_csv('/kaggle/input/vid2img-nfl-impacts/impact_video_labels', index_col=0)

In [ ]:
df.head()

In [ ]:
df[df['impactType'] != '0']

In [ ]:
df[df['impact']==2]['impactType'].value_counts()

In [ ]:
df[df['impact']==1]['impactType'].value_counts()

In [ ]:
X = df[['left','width','top','height']].copy()
X = X.to_numpy()

In [ ]:
y = df['impact'].copy()
y= y.apply(lambda x: 0 if x==1 else 1)
y = y.to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_test, label=y_test)

In [ ]:
classification = True
# instantiate params
params = {}

# general params
general_params = {'silent': 1}
params.update(general_params)

# booster params
n_gpus = 1  # change this to -1 to use all GPUs available or 0 to use the CPU
booster_params = {}

if n_gpus != 0:
    booster_params['tree_method'] = 'gpu_hist'
    booster_params['n_gpus'] = n_gpus   
params.update(booster_params)

# learning task params
learning_task_params = {}
if classification:
    learning_task_params['eval_metric'] = 'auc'
    learning_task_params['objective'] = 'binary:logistic'
else:
    learning_task_params['eval_metric'] = 'rmse'
    learning_task_params['objective'] = 'reg:squarederror'
params.update(learning_task_params)
print(params)

In [ ]:

# model training settings
evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
num_round = 100

In [ ]:
%%time

bst = xgb.train(params, dtrain, num_round, evallist)